In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-6.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...  dam  moa  \
0    prop-6      452    2    2    0    5    9     1   1   4  ...  0.0    0   
1    prop-6      452    3    1    0    1    3     3   1   0  ...  0.0    0   
2    prop-6      452   13    2    0    8   17    78   0   8  ...  0.0    0   
3    prop-6      452    2    2    0    5    7     1   1   4  ...  0.0    0   
4    prop-6      452   23    1    0   10   23   253   2   8  ...  0.0    0   
..      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...  ...  ...   
655  prop-6      454    9    2    0   10   20    36   5  10  ...  0.0    0   
656  prop-6      454    6    2    0    7   15    15   2   7  ...  0.0    0   
657  prop-6      454    6    1    0    5   29    11   2   4  ...  0.0    0   
658  prop-6      454    6    1    0    3   31     1   1   2  ...  1.0    1   
659  prop-6      454   29    1    0   11   50   180  10   1  ...  1.0    0   

          mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0    0.909091  0.750000   1    1  23.500000       1  0.5000    1  
1    0.000000  1.000000   0    0   0.000000       1  1.0000    0  
2    0.586207  1.000000   1    3   2.769231       1  0.9231    0  
3    0.909091  0.750000   1    2   7.500000       1  0.5000    0  
4    0.000000  0.255435   0    0   0.000000       1  1.0000    0  
..        ...       ...  ..  ...        ...     ...     ...  ...  
655  0.200000  0.377778   0    0   7.888889       1  0.8889    0  
656  0.285714  0.400000   0    0   6.666667       1  0.8333    0  
657  0.000000  0.400000   0    0  26.666667       4  1.5000    0  
658  0.000000  0.333333   0    0  32.500000       3  1.3333    0  
659  0.000000  0.226601   1    1  13.206897       7  1.2759    0  

[660 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 452]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...  dam  moa  \
0    prop-6      452    2    2    0    5    9     1   1   4  ...  0.0    0   
1    prop-6      452    3    1    0    1    3     3   1   0  ...  0.0    0   
2    prop-6      452   13    2    0    8   17    78   0   8  ...  0.0    0   
3    prop-6      452    2    2    0    5    7     1   1   4  ...  0.0    0   
4    prop-6      452   23    1    0   10   23   253   2   8  ...  0.0    0   
..      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...  ...  ...   
251  prop-6      452    7    1    0    5    7    21   3   2  ...  0.0    0   
252  prop-6      452    2    1    0    3    2     1   2   1  ...  0.0    0   
253  prop-6      452    4    1    0    3    8     0   2   1  ...  1.0    0   
254  prop-6      452    6    1    0    6    6    15   4   2  ...  0.0    0   
255  prop-6      452    2    2    0    4    5     1   0   4  ...  0.0    0   

          mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0    0.909091  0.750000   1    1  23.500000       1  0.5000    1  
1    0.000000  1.000000   0    0   0.000000       1  1.0000    0  
2    0.586207  1.000000   1    3   2.769231       1  0.9231    0  
3    0.909091  0.750000   1    2   7.500000       1  0.5000    0  
4    0.000000  0.255435   0    0   0.000000       1  1.0000    0  
..        ...       ...  ..  ...        ...     ...     ...  ...  
251  0.000000  0.535714   0    0   0.000000       1  1.0000    0  
252  0.000000  0.750000   0    0   0.000000       1  1.0000    0  
253  0.000000  0.500000   0    0   6.000000       1  0.7500    0  
254  0.000000  0.333333   0    0   0.000000       1  1.0000    0  
255  0.666667  0.750000   0    0   4.500000       1  0.5000    0  

[256 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc  dam  moa  \
116    4    2    0    5    9     6   0   5    4  2.000000   38  0.0    0   
220    5    1    0    7   14     0   3   4    5  0.333333  115  1.0    0   
110    7    2    0   11   17    21   4  11    7  2.000000   80  0.0    0   
194   13    2    0    9   19    78   0   9   13  2.000000   65  0.0    0   
109    5    2    0   10   42     2   0  10    1  1.125000  262  1.0    1   
..   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...  ...  ...   
19     2    1    0    3    7     1   2   1    1  2.000000   32  0.0    0   
241    9    1    0    5   13    20   4   1    9  0.791667   99  1.0    1   
162    6    1    0   20   47    13   0  20    4  1.160000  274  1.0    1   
35     7    1   15   20   10    11  15   5    5  0.777778   39  1.0    2   
195    4    2    0   11   29     2   0  11    1  1.214286  247  1.0    1   

          mfa       cam  ic  cbm        amc  max_cc  avg_cc  
116  0.400000  0.400000   0    0   8.500000       1  0.7500  
220  0.000000  0.566667   0    0  21.400000       4  2.0000  
110  0.250000  0.309524   0    0  10.428571       1  0.8571  
194  0.586207  1.000000   1    3   4.000000       1  0.9231  
109  0.800000  0.562500   1    1  49.000000       6  1.6000  
..        ...       ...  ..  ...        ...     ...     ...  
19   0.000000  0.500000   0    0  15.000000       2  1.0000  
241  0.000000  0.288889   1    1   9.666667       7  1.6667  
162  0.000000  0.416667   0    0  43.000000       2  1.1667  
35   0.000000  0.476190   0    0   4.142857       1  0.8571  
195  0.857143  0.583333   1    1  57.250000       8  2.2500  

[204 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  moa  \
101   12    1    0   17   50    18  12   7    7  0.727273  253  1.000000    0   
239   10    1    0    1   23     9   1   0    9  0.755556  155  1.000000    0   
98    14    2    0    8   19    91   0   8   14  2.000000   86  0.000000    0   
230    5    1    4    4    9     0   4   0    4  0.375000   35  1.000000    0   
55     3    5    0   16    6     3  15   1    2  1.500000   16  1.000000    0   
51    16    1    0   20   65    88   0  20    9  0.857143  367  1.000000    6   
159   15    1    0   23   27    51  23   1   15  0.767857  169  1.000000    0   
28     5    2    0   31    8     6  30   1    3  0.875000   79  0.333333    5   
170   33    1    0   27   66   270  22   5   33  0.869318  509  1.000000    3   
185   15    1    0   13   42    41   0  13   15  0.846939  221  0.857143    6   
173   11    1    0    7   22    17   7   0   10  0.814286  150  1.000000    0   
200    2    2    0    5    9     1   1   4    1  2.000000   49  0.000000    0   
209   15    1    0    3   21    63   3   1   15  0.828571  152  1.000000    1   
38    12    1    0   15   42    16   0  15   10  0.757576  354  1.000000    5   
53    13    2    0    8   17    78   0   8   13  2.000000   49  0.000000    0   
68     7    2    0   13   39     9   0  13    1  1.062500  366  1.000000    4   
5     13    1    0    3   18    26   3   0   13  0.750000  134  1.000000    0   
81     3    2    0    8   14     3   1   7    3  2.000000  103  0.000000    0   
160    5    1    0   27   15    10  26   1    4  2.000000   89  0.000000    0   
138   13    2    0    8   17    78   0   8   13  2.000000   49  0.000000    0   
169    8    1    0    5   13    10   4   1    8  0.619048   56  1.000000    0   
237   14    2    0    7   17    91   0   7   14  2.000000   53  0.000000    0   
157    3    2    0    6    8     3   1   6    3  2.000000   21  0.000000    0   
73     4    1    0    6   15     0   1   5    3  0.666667   68  1.000000    1   
229    4    1    0    2    9     0   2   0    4  0.500000   45  1.000000    0   
33    16    1    0    5   41     0   2   4   14  0.775000  254  0.750000    1   
8      3    2    0    8   14     3   1   7    3  2.000000   55  0.000000    0   
1      3    1    0    1    3     3   1   0    3  2.000000    3  0.000000    0   
9     13    2    0    8   17    78   0   8   13  2.000000   49  0.000000    0   
135    8    1    0    9    9    14   8   1    7  0.857143   46  1.000000    1   
233   29    1    0   11   50   180  10   1   26  0.726190  418  1.000000    0   
126   13    2    0    8   17    78   0   8   13  2.000000   49  0.000000    0   
193    6    2    0    2   14    11   1   1    4  0.800000  105  0.400000    4   
4     23    1    0   10   23   253   2   8   23  2.000000   23  0.000000    0   
60     3    2    0    8   15     3   1   7    3  2.000000   63  0.000000    0   
92     9    2    0   10   20    36   4  10    9  2.000000   77  0.000000    0   
27     6    2    0   11   43     1   0  11    2  1.000000  243  1.000000    1   
97     9    1    0    7   16     6   7   0    8  0.708333  104  1.000000    0   
31     2    1    0    2    2     1   2   0    2  2.000000    2  0.000000    0   
17     8    2    0   23   61     6   1  22    2  0.761905  413  1.000000    1   
201    3    2    0    8   14     3   1   7    3  2.000000  103  0.000000    0   
255    2    2    0    4    5     1   0   4    2  2.000000   11  0.000000    0   
10     2    2    0    7    6     1   6   1    2  2.000000   16  0.000000    0   
164    4    1    0    0    5     0   0   0    4  1.000000   23  0.333333    0   
139    2    2    0    5    7     1   1   4    1  2.000000   17  0.000000    0   
90     3    1    0    2   13     3   2   0    1  2.000000   90  0.000000    0   
177    3    2    0    8   14     3   1   7    3  2.000000   63  0.000000    0   
152    3    3    0    3    4     1   2   1    3  1.000000   15  1.000000    0   
191    6    1    0    1   13    15  

In [9]:
y_train = train['bug']
y_test = test['bug']

In [10]:
y_train

116    0
220    0
110    0
194    0
109    1
      ..
19     0
241    0
162    0
35     0
195    1
Name: bug, Length: 204, dtype: int64

In [11]:
y_test

101    0
239    0
98     0
230    0
55     0
51     0
159    0
28     0
170    0
185    0
173    0
200    1
209    0
38     0
53     0
68     0
5      0
81     0
160    0
138    0
169    0
237    0
157    0
73     0
229    0
33     1
8      0
1      0
9      0
135    0
233    0
126    0
193    0
4      0
60     0
92     0
27     0
97     0
31     0
17     1
201    0
255    0
10     0
164    0
139    0
90     0
177    0
152    0
191    0
180    1
203    0
251    0
Name: bug, dtype: int64

In [12]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [13]:
x_train_np

array([[ 4.        ,  2.        ,  0.        , ...,  8.5       ,
         1.        ,  0.75      ],
       [ 5.        ,  1.        ,  0.        , ..., 21.4       ,
         4.        ,  2.        ],
       [ 7.        ,  2.        ,  0.        , ..., 10.42857143,
         1.        ,  0.8571    ],
       ...,
       [ 6.        ,  1.        ,  0.        , ..., 43.        ,
         2.        ,  1.1667    ],
       [ 7.        ,  1.        , 15.        , ...,  4.14285714,
         1.        ,  0.8571    ],
       [ 4.        ,  2.        ,  0.        , ..., 57.25      ,
         8.        ,  2.25      ]])

In [14]:
x_test_np

array([[12.        ,  1.        ,  0.        , ..., 19.75      ,
         5.        ,  1.5833    ],
       [10.        ,  1.        ,  0.        , ..., 14.        ,
         3.        ,  1.        ],
       [14.        ,  2.        ,  0.        , ...,  5.14285714,
         1.        ,  0.9286    ],
       ...,
       [18.        ,  1.        ,  0.        , ..., 31.94444444,
         8.        ,  2.6667    ],
       [ 3.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [ 7.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ]])

In [15]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [16]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [17]:
train_x

array([[[ 4.        ,  2.        ,  0.        , ...,  8.5       ,
          1.        ,  0.75      ]],

       [[ 5.        ,  1.        ,  0.        , ..., 21.4       ,
          4.        ,  2.        ]],

       [[ 7.        ,  2.        ,  0.        , ..., 10.42857143,
          1.        ,  0.8571    ]],

       ...,

       [[ 6.        ,  1.        ,  0.        , ..., 43.        ,
          2.        ,  1.1667    ]],

       [[ 7.        ,  1.        , 15.        , ...,  4.14285714,
          1.        ,  0.8571    ]],

       [[ 4.        ,  2.        ,  0.        , ..., 57.25      ,
          8.        ,  2.25      ]]])

In [18]:
test_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0])

In [19]:
def define_model():
    # Designing and initializing the model.
    model = Sequential()
    model.add(LSTM(200, return_sequences=True))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(50, dropout = 0.2, return_sequences=True))
    model.add(LSTM(20, return_sequences=False))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0005) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )
    return model

In [20]:
y = list(test_y)

In [21]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(10):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/propv452/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(test_x, test_y)
    total_scores.append(score) 
    predictions = model.predict(test_x)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
3/3 [==============================] - 8s 176ms/step - loss: 0.1699 - mse: 0.1699 - mae: 0.1552 - root_mean_squared_error: 0.4122 - mean_squared_logarithmic_error: 0.0743
Epoch 2/100
3/3 [==============================] - 0s 17ms/step - loss: 0.1670 - mse: 0.1670 - mae: 0.1625 - root_mean_squared_error: 0.4087 - mean_squared_logarithmic_error: 0.0725
Epoch 3/100
3/3 [==============================] - 0s 16ms/step - loss: 0.1644 - mse: 0.1644 - mae: 0.1691 - root_mean_squared_error: 0.4055 - mean_squared_logarithmic_error: 0.0709
Epoch 4/100
3/3 [==============================] - 0s 16ms/step - loss: 0.1615 - mse: 0.1615 - mae: 0.1763 - root_mean_squared_error: 0.4018 - mean_squared_logarithmic_error: 0.0691
Epoch 5/100
3/3 [==============================] - 0s 19ms/step - loss: 0.1590 - mse: 0.1590 - mae: 0.1845 - root_mean_squared_error: 0.3987 - mean_squared_logarithmic_error: 0.0678
Epoch 6/100
3/3 [==============================] - 0s 18ms/step - loss: 0.1557 

3/3 [==============================] - 0s 17ms/step - loss: 0.1018 - mse: 0.1018 - mae: 0.2043 - root_mean_squared_error: 0.3190 - mean_squared_logarithmic_error: 0.0471
Epoch 89/100
3/3 [==============================] - 0s 20ms/step - loss: 0.1032 - mse: 0.1032 - mae: 0.1855 - root_mean_squared_error: 0.3212 - mean_squared_logarithmic_error: 0.0449
Epoch 90/100
3/3 [==============================] - 0s 16ms/step - loss: 0.1073 - mse: 0.1073 - mae: 0.1808 - root_mean_squared_error: 0.3275 - mean_squared_logarithmic_error: 0.0460
Epoch 91/100
3/3 [==============================] - 0s 18ms/step - loss: 0.1064 - mse: 0.1064 - mae: 0.1763 - root_mean_squared_error: 0.3261 - mean_squared_logarithmic_error: 0.0453
Epoch 92/100
3/3 [==============================] - 0s 16ms/step - loss: 0.1059 - mse: 0.1059 - mae: 0.1761 - root_mean_squared_error: 0.3254 - mean_squared_logarithmic_error: 0.0450
Epoch 93/100
3/3 [==============================] - 0s 16ms/step - loss: 0.1041 - mse: 0.1041 - ma

3/3 [==============================] - 0s 16ms/step - loss: 0.1029 - mse: 0.1029 - mae: 0.2069 - root_mean_squared_error: 0.3208 - mean_squared_logarithmic_error: 0.0486
Epoch 75/100
3/3 [==============================] - 0s 16ms/step - loss: 0.0980 - mse: 0.0980 - mae: 0.1853 - root_mean_squared_error: 0.3131 - mean_squared_logarithmic_error: 0.0446
Epoch 76/100
3/3 [==============================] - 0s 19ms/step - loss: 0.1038 - mse: 0.1038 - mae: 0.1805 - root_mean_squared_error: 0.3222 - mean_squared_logarithmic_error: 0.0464
Epoch 77/100
3/3 [==============================] - 0s 17ms/step - loss: 0.1136 - mse: 0.1136 - mae: 0.1797 - root_mean_squared_error: 0.3370 - mean_squared_logarithmic_error: 0.0501
Epoch 78/100
3/3 [==============================] - 0s 16ms/step - loss: 0.1162 - mse: 0.1162 - mae: 0.1683 - root_mean_squared_error: 0.3409 - mean_squared_logarithmic_error: 0.0492
Epoch 79/100
3/3 [==============================] - 0s 21ms/step - loss: 0.1125 - mse: 0.1125 - ma

3/3 [==============================] - 0s 16ms/step - loss: 0.1205 - mse: 0.1205 - mae: 0.1905 - root_mean_squared_error: 0.3471 - mean_squared_logarithmic_error: 0.0503
Epoch 61/100
3/3 [==============================] - 0s 18ms/step - loss: 0.1186 - mse: 0.1186 - mae: 0.1896 - root_mean_squared_error: 0.3444 - mean_squared_logarithmic_error: 0.0494
Epoch 62/100
3/3 [==============================] - 0s 18ms/step - loss: 0.1172 - mse: 0.1172 - mae: 0.1969 - root_mean_squared_error: 0.3423 - mean_squared_logarithmic_error: 0.0492
Epoch 63/100
3/3 [==============================] - 0s 16ms/step - loss: 0.1157 - mse: 0.1157 - mae: 0.2068 - root_mean_squared_error: 0.3402 - mean_squared_logarithmic_error: 0.0497
Epoch 64/100
3/3 [==============================] - 0s 16ms/step - loss: 0.1155 - mse: 0.1155 - mae: 0.2199 - root_mean_squared_error: 0.3399 - mean_squared_logarithmic_error: 0.0514
Epoch 65/100
3/3 [==============================] - 0s 18ms/step - loss: 0.1153 - mse: 0.1153 - ma

3/3 [==============================] - 0s 18ms/step - loss: 0.1208 - mse: 0.1208 - mae: 0.2478 - root_mean_squared_error: 0.3475 - mean_squared_logarithmic_error: 0.0569
Epoch 47/100
3/3 [==============================] - 0s 14ms/step - loss: 0.1166 - mse: 0.1166 - mae: 0.2401 - root_mean_squared_error: 0.3414 - mean_squared_logarithmic_error: 0.0546
Epoch 48/100
3/3 [==============================] - 0s 16ms/step - loss: 0.1168 - mse: 0.1168 - mae: 0.2194 - root_mean_squared_error: 0.3418 - mean_squared_logarithmic_error: 0.0515
Epoch 49/100
3/3 [==============================] - 0s 22ms/step - loss: 0.1178 - mse: 0.1178 - mae: 0.1990 - root_mean_squared_error: 0.3433 - mean_squared_logarithmic_error: 0.0499
Epoch 50/100
3/3 [==============================] - 0s 15ms/step - loss: 0.1160 - mse: 0.1160 - mae: 0.1941 - root_mean_squared_error: 0.3406 - mean_squared_logarithmic_error: 0.0494
Epoch 51/100
3/3 [==============================] - 0s 16ms/step - loss: 0.1153 - mse: 0.1153 - ma

...

3/3 [==============================] - 0s 18ms/step - loss: 0.0783 - mse: 0.0783 - mae: 0.1341 - root_mean_squared_error: 0.2798 - mean_squared_logarithmic_error: 0.0349
Epoch 99/100
3/3 [==============================] - 0s 15ms/step - loss: 0.0851 - mse: 0.0851 - mae: 0.1352 - root_mean_squared_error: 0.2918 - mean_squared_logarithmic_error: 0.0367
Epoch 100/100
2/2 [==============================] - 1s 6ms/step - loss: 0.0675 - mse: 0.0675 - mae: 0.1054 - root_mean_squared_error: 0.2598 - mean_squared_logarithmic_error: 0.0315
score of 5th run is: 0.06749799847602844, 0.06749799847602844, 0.10540835559368134, 0.25980377197265625, 0.03149453550577164
Run no: 6
Epoch 1/100
3/3 [==============================] - 8s 293ms/step - loss: 0.1708 - mse: 0.1708 - mae: 0.1546 - root_mean_squared_error: 0.4133 - mean_squared_logarithmic_error: 0.0750
Epoch 2/100
3/3 [==============================] - 0s 15ms/step - loss: 0.1669 - mse: 0.1669 - mae: 0.1646 - root_mean_squared_error: 0.4085 - mea

3/3 [==============================] - 0s 17ms/step - loss: 0.1105 - mse: 0.1105 - mae: 0.2222 - root_mean_squared_error: 0.3324 - mean_squared_logarithmic_error: 0.0529
Epoch 77/100
3/3 [==============================] - 0s 17ms/step - loss: 0.1071 - mse: 0.1071 - mae: 0.2044 - root_mean_squared_error: 0.3272 - mean_squared_logarithmic_error: 0.0489
Epoch 78/100
3/3 [==============================] - 0s 16ms/step - loss: 0.1072 - mse: 0.1072 - mae: 0.1890 - root_mean_squared_error: 0.3275 - mean_squared_logarithmic_error: 0.0470
Epoch 79/100
3/3 [==============================] - 0s 15ms/step - loss: 0.1070 - mse: 0.1070 - mae: 0.1787 - root_mean_squared_error: 0.3272 - mean_squared_logarithmic_error: 0.0455
Epoch 80/100
3/3 [==============================] - 0s 17ms/step - loss: 0.1052 - mse: 0.1052 - mae: 0.1747 - root_mean_squared_error: 0.3243 - mean_squared_logarithmic_error: 0.0444
Epoch 81/100
3/3 [==============================] - 0s 16ms/step - loss: 0.1042 - mse: 0.1042 - ma

3/3 [==============================] - 0s 6ms/step - loss: 0.1034 - mse: 0.1034 - mae: 0.1677 - root_mean_squared_error: 0.3215 - mean_squared_logarithmic_error: 0.0451
Epoch 64/100
3/3 [==============================] - 0s 7ms/step - loss: 0.1086 - mse: 0.1086 - mae: 0.1547 - root_mean_squared_error: 0.3295 - mean_squared_logarithmic_error: 0.0453
Epoch 65/100
3/3 [==============================] - 0s 7ms/step - loss: 0.1138 - mse: 0.1138 - mae: 0.1494 - root_mean_squared_error: 0.3374 - mean_squared_logarithmic_error: 0.0465
Epoch 66/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1130 - mse: 0.1130 - mae: 0.1469 - root_mean_squared_error: 0.3362 - mean_squared_logarithmic_error: 0.0462
Epoch 67/100
3/3 [==============================] - 0s 7ms/step - loss: 0.1067 - mse: 0.1067 - mae: 0.1532 - root_mean_squared_error: 0.3266 - mean_squared_logarithmic_error: 0.0447
Epoch 68/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1056 - mse: 0.1056 - mae: 0.1

3/3 [==============================] - 0s 7ms/step - loss: 0.1214 - mse: 0.1214 - mae: 0.2248 - root_mean_squared_error: 0.3485 - mean_squared_logarithmic_error: 0.0566
Epoch 51/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1197 - mse: 0.1197 - mae: 0.2164 - root_mean_squared_error: 0.3459 - mean_squared_logarithmic_error: 0.0550
Epoch 52/100
3/3 [==============================] - 0s 7ms/step - loss: 0.1173 - mse: 0.1173 - mae: 0.2051 - root_mean_squared_error: 0.3424 - mean_squared_logarithmic_error: 0.0527
Epoch 53/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1156 - mse: 0.1156 - mae: 0.2038 - root_mean_squared_error: 0.3400 - mean_squared_logarithmic_error: 0.0522
Epoch 54/100
3/3 [==============================] - 0s 7ms/step - loss: 0.1131 - mse: 0.1131 - mae: 0.2054 - root_mean_squared_error: 0.3363 - mean_squared_logarithmic_error: 0.0517
Epoch 55/100
3/3 [==============================] - 0s 7ms/step - loss: 0.1134 - mse: 0.1134 - mae: 0.2

3/3 [==============================] - 0s 6ms/step - loss: 0.1303 - mse: 0.1303 - mae: 0.2294 - root_mean_squared_error: 0.3610 - mean_squared_logarithmic_error: 0.0572
Epoch 37/100
3/3 [==============================] - 0s 7ms/step - loss: 0.1293 - mse: 0.1293 - mae: 0.2254 - root_mean_squared_error: 0.3596 - mean_squared_logarithmic_error: 0.0565
Epoch 38/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1282 - mse: 0.1282 - mae: 0.2272 - root_mean_squared_error: 0.3581 - mean_squared_logarithmic_error: 0.0563
Epoch 39/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1277 - mse: 0.1277 - mae: 0.2296 - root_mean_squared_error: 0.3574 - mean_squared_logarithmic_error: 0.0565
Epoch 40/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1284 - mse: 0.1284 - mae: 0.2271 - root_mean_squared_error: 0.3583 - mean_squared_logarithmic_error: 0.0565
Epoch 41/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1275 - mse: 0.1275 - mae: 0.2

In [22]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/propv452/'

In [23]:
total_training_time

[18.880354166030884,
 16.768449068069458,
 19.16733407974243,
 17.688753366470337,
 18.546361446380615,
 19.051985502243042,
 18.13500690460205,
 10.595309972763062,
 8.826001405715942,
 8.757495641708374]

In [24]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 5.21390171845754


In [25]:
total_training_loss_history

[{'loss': [0.1698751449584961,
   0.16703590750694275,
   0.164446622133255,
   0.1614561676979065,
   0.15895013511180878,
   0.155707448720932,
   0.1536446064710617,
   0.15041562914848328,
   0.14824683964252472,
   0.14598214626312256,
   0.14458294212818146,
   0.14407750964164734,
   0.14358536899089813,
   0.14171205461025238,
   0.1425623744726181,
   0.14189594984054565,
   0.1407962292432785,
   0.1408088505268097,
   0.14077594876289368,
   0.14086560904979706,
   0.14017485082149506,
   0.1392849087715149,
   0.13779015839099884,
   0.13614578545093536,
   0.1351076364517212,
   0.1333625167608261,
   0.13254959881305695,
   0.13175396621227264,
   0.13252942264080048,
   0.13096337020397186,
   0.13027065992355347,
   0.12801378965377808,
   0.12963296473026276,
   0.12876497209072113,
   0.12666106224060059,
   0.12672708928585052,
   0.12646056711673737,
   0.12676039338111877,
   0.1259358823299408,
   0.12478624284267426,
   0.1243499368429184,
   0.12390155345201492,

In [26]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(10):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 0.03248815461993217
avg mae of training: 0.05556890368461609


In [27]:
total_scores

[[0.08558444678783417,
  0.08558444678783417,
  0.20221446454524994,
  0.29254820942878723,
  0.05076069012284279],
 [0.06965450942516327,
  0.06965450942516327,
  0.14061062037944794,
  0.2639214098453522,
  0.0365397073328495],
 [0.0681808739900589,
  0.0681808739900589,
  0.1509736329317093,
  0.2611146867275238,
  0.036756619811058044],
 [0.06390810757875443,
  0.06390810757875443,
  0.15001176297664642,
  0.25280052423477173,
  0.036139246076345444],
 [0.06450433284044266,
  0.06450433284044266,
  0.1247001588344574,
  0.2539770305156708,
  0.032494738698005676],
 [0.06749799847602844,
  0.06749799847602844,
  0.10540835559368134,
  0.25980377197265625,
  0.03149453550577164],
 [0.07539656013250351,
  0.0753965675830841,
  0.18436136841773987,
  0.2745843529701233,
  0.04160517081618309],
 [0.06661291420459747,
  0.06661291420459747,
  0.110213503241539,
  0.25809478759765625,
  0.03270067274570465],
 [0.06294329464435577,
  0.06294329464435577,
  0.11454538255929947,
  0.25088500

In [1]:
a = [[0.08558444678783417,
  0.08558444678783417,
  0.20221446454524994,
  0.29254820942878723,
  0.05076069012284279],
 [0.06965450942516327,
  0.06965450942516327,
  0.14061062037944794,
  0.2639214098453522,
  0.0365397073328495],
 [0.0681808739900589,
  0.0681808739900589,
  0.1509736329317093,
  0.2611146867275238,
  0.036756619811058044],
 [0.06390810757875443,
  0.06390810757875443,
  0.15001176297664642,
  0.25280052423477173,
  0.036139246076345444],
 [0.06450433284044266,
  0.06450433284044266,
  0.1247001588344574,
  0.2539770305156708,
  0.032494738698005676],
 [0.06749799847602844,
  0.06749799847602844,
  0.10540835559368134,
  0.25980377197265625,
  0.03149453550577164],
 [0.07539656013250351,
  0.0753965675830841,
  0.18436136841773987,
  0.2745843529701233,
  0.04160517081618309],
 [0.06661291420459747,
  0.06661291420459747,
  0.110213503241539,
  0.25809478759765625,
  0.03270067274570465],
 [0.06294329464435577,
  0.06294329464435577,
  0.11454538255929947,
  0.250885009765625,
  0.032027069479227066],
 [0.0715637356042862,
  0.0715637356042862,
  0.15912841260433197,
  0.2675139904022217,
  0.03912752494215965]]

In [2]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])

In [4]:
sum(test_mae)/10

0.14421676620841026

In [31]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(10):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 0.02319489245613416
avg mae: 0.048072255402803424
avg are: 0.025641026596228283


In [32]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.09638415649533272
median testing mse: 0.06783943623304367


In [33]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.16753768920898438
median testing mae: 0.14531119167804718


In [34]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.07692307978868484


In [35]:
test_ARE

[0.07692308,
 0.07692308,
 0.07692308,
 0.07692308,
 0.07692308,
 0.07692308,
 0.07692308,
 0.07692308,
 0.07692308,
 0.07692308]